In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:


def cal_bias_prop(bias_val_list):
    num_bias = 0
    num_unbias = 0
    for bias_val in bias_val_list:
        if bias_val >= 0.5:
            num_bias += 1
        else:
            num_unbias += 1
    return 0 if num_bias + num_unbias == 0 else num_bias / (num_bias + num_unbias)

def generate_choice_arr(choice_path, history_path, num_hist, write_path):
    bias_prop_user_all_arr = []
    
    with open(history_path, "rb") as f:
        history_list_ = pickle.load(f)
    
    with open(choice_path, "rb") as f:
        choice_info = pickle.load(f)
        
    for jdx, (uid, user) in enumerate(choice_info.items()):
        
        user_bias_prop_user_all_arr = []

        step_list = []
        choice_bias_list = []

        bias_dict = user["bias"]
        for step, bias_list in bias_dict.items():
            step_list.append(step)
            choice_bias_list.append(bias_list)

        num_step = len(choice_bias_list)

        history_list = history_list_[jdx][:num_hist]
        user_bias_prop_user_all_arr.append(cal_bias_prop(history_list))
        for idx in range(num_step):
            total_choice_list = []
            for bias_list in choice_bias_list[:idx + 1]:
                total_choice_list.extend(bias_list)
            total_choice_list = total_choice_list[::-1]  # reversed by time
            total_list = total_choice_list + history_list

            user_bias_prop_user_all_arr.append(cal_bias_prop(total_list))

        bias_prop_user_all_arr.append(user_bias_prop_user_all_arr)
    
    bias_prop_user_all_arr = np.asarray(bias_prop_user_all_arr)

    with open(write_path, "wb") as f:
        np.save(f, bias_prop_user_all_arr)


In [ ]:

def show_dynamic_res(user_bias_prop_arr_path, 
                     algorithm_name, 
                     fig_save_path, 
                     legend_save_path="",
                     separate_legend=False,
                     strategy_name=""):
    bias_arr = np.array([5, 10, 15, 20, 25, 30, 35, 40])
    unbias_arr = np.array([45, 40, 35, 30, 25, 20, 15, 10])
    bias_prop_arr = bias_arr/(bias_arr+unbias_arr)

#     random_label_list = [f'random-{bias_prop_val*100:.0f}%' for bias_prop_val in bias_prop_arr]
#     algorithm_label_list = [f'{algorithm_name}-{bias_prop_val*100:.0f}%' for bias_prop_val in bias_prop_arr]
    random_label_list = [f'{bias_prop_val*100:.0f}%' for bias_prop_val in bias_prop_arr]
    algorithm_label_list = [f'{bias_prop_val*100:.0f}%' for bias_prop_val in bias_prop_arr]

    # env setting
    color_list = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red',
                  'tab:purple', 'tab:brown', 'tab:pink', 'tab:cyan']
    plt.figure(figsize=(10, 10))
    plt.yticks(fontproperties='Times New Roman', size=15, weight='bold')
    plt.xticks(fontproperties='Times New Roman', size=15, weight='bold')
#     plt.rcParams['font.sans-serif'] = ['Times News Roman']
    plt.ylim((0, 100))
    plt.xlim((0, 66))

    with open(user_bias_prop_arr_path, "rb") as f:
        bias_prop_user_all_arr = np.load(f)
    for jdx, idx in enumerate(range(0, 2000, 250)):
        plt.plot(np.average(bias_prop_user_all_arr[idx: idx + 250], axis=0)*100,
                 color=color_list[jdx], )#label=algorithm_label_list[jdx])
    random_arr = [[bias_prop_val, ] * 66 for bias_prop_val in bias_prop_arr]
    for jdx, idx in enumerate(range(0, 2000, 250)):
        plt.plot(np.array(random_arr[jdx])*100,
                 color=color_list[jdx], label=random_label_list[jdx], linestyle='dashed', )

#     plt.grid()
    if strategy_name:
        plt.title(strategy_name, fontproperties='Times New Roman')
        
    if separate_legend:
        pass
#         plt.savefig(fig_save_path, format='pdf')
        
# #         legend = plt.legend(bbox_to_anchor=(1.0, 0.23), ncol=1, )
#         legend = plt.legend(ncol=8, fontsize=15)
#         def export_legend(legend, filename="legend.png"):
#             fig  = legend.figure
#             fig.canvas.draw()
#             bbox  = legend.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
#             fig.savefig(filename, format='pdf', bbox_inches=bbox)
#         export_legend(legend=legend,
#                       filename=legend_save_path)
    else:
#         print(fig_save_path)
        plt.legend(bbox_to_anchor=(1.0, 0.23), ncol=2, )
        plt.savefig(fig_save_path, format='png')
    


In [ ]:

choice_dir_root=""
user_bp_dir_root=""
fig_dir_root=""
history_path=""
random_user_bias_prop_arr_path=""

folder_list = ['nrms', 'plm', 'npa', 'fim', 'naml']
algorithm_name_list = ['NRMS', 'PLM.', 'NPA', 'FIM', 'NAML']
strategy_name_list = ['Equal-Rec', 'Top-K-Rec', 'Log-Rank-Rec', 
                      'Supportive-User-Awareness', 'Adversial-User-Awareness',
                      'LRR-SUA', 'LRR-AUA']
for folder_name, algorithm_name in zip(folder_list, algorithm_name_list):

    for s_idx in range(1, 8):
        choice_path=os.path.join(choice_dir_root, f"{folder_name}_constant/{folder_name}_dynamic_s{s_idx}_user_choice.pkl")
        user_bp_dir_path=os.path.join(user_bp_dir_root, f"{folder_name}/")
        algorithm_name=algorithm_name
        fig_dir_path=os.path.join(fig_dir_root, f"{folder_name}/")

        user_prefix=choice_path.split("/")[-1].split("_user_choice.pkl")[0]
        num_hist=50

        os.makedirs(user_bp_dir_path, exist_ok=True)
        user_bp_path=os.path.join(user_bp_dir_path, f"{user_prefix}.npy")
        generate_choice_arr(choice_path=choice_path, 
                            history_path=history_path, 
                            num_hist=num_hist, 
                            write_path=user_bp_path)

        os.makedirs(fig_dir_path, exist_ok=True)
        fig_path=os.path.join(fig_dir_path, f"{user_prefix}.pdf")
        legend_fig_path=os.path.join(fig_dir_path, f"{user_prefix}_legend.pdf")
        show_dynamic_res(user_bias_prop_arr_path=user_bp_path,
                         algorithm_name=algorithm_name,
                         fig_save_path=fig_path,
                         legend_save_path=legend_fig_path,
                         separate_legend=True,
                         strategy_name=None)
#                          strategy_name=strategy_name_list[s_idx-1])







In [ ]:
# legend
from matplotlib.lines import Line2D
legend_file_name_path=""
user_group_list = ['10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%']
color_list = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red',
             'tab:purple', 'tab:brown', 'tab:pink', 'tab:cyan']
custom_lines = []
for color_name in color_list:
    custom_lines.append(Line2D([0], [0], color=color_name, lw=4))

fig, ax = plt.subplots()
# lines = ax.plot(data)
legend = ax.legend(custom_lines, user_group_list, bbox_to_anchor=(1.0, 1.23), ncol=8, fontsize=15, )
def export_legend(legend, filename="legend.png"):
    fig  = legend.figure
    fig.canvas.draw()
    bbox  = legend.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(filename, dpi='figure', format='pdf', bbox_inches=bbox)
export_legend(legend=legend,
              filename=legend_file_name_path)